In [1]:
ls

Data Generation.ipynb  dowloaddata.sh*        ptb.valid.txt
Line Generation.ipynb  dumps/                 ptb.vocab.json
README.md              figs/                  requirements.txt
__pycache__/           inference.py           simple-examples/
bin/                   model.py               simple-examples.tgz
data/                  ptb.py                 train.py
data-copy/             ptb.train.txt          utils.py


In [2]:
ls data

clean_poems.csv      glove.840B.300d.txt  ptb.vocab.json
final_poems.csv      ptb.train.txt        ultimate_poems.csv
glove.6B.300d.txt    ptb.valid.txt


In [3]:
import pandas as pd
import json
import spacy

In [4]:
import torch
# prepare glove vectors
words = set()
word_to_vec = {}
word_to_id = {}
id_to_word = {}
glove_dim = 300
glove_filename = "data/glove.840B.300d.txt" # "./glove/glove.6B.300d.txt"
with open(glove_filename) as file:
    count = 0
    for line in file:
        tokens = line.split()
        word, vals = tokens[0], tokens[-glove_dim:]
        if word not in words:
            words.add(word)
            #word_to_vec[word] = torch.Tensor([float(val) for val in vals])
            word_to_id[word] = count
            id_to_word[count] = word
            count += 1
print(len(words))
print(id_to_word[1])

2195884
.


In [5]:
nlp = spacy.load('en')
filename = "data/final_poems.csv"
poems = pd.read_csv(filename)

In [6]:
from nltk.tokenize import sent_tokenize, word_tokenize
poem_tokens = [word_tokenize(poem) for poem in poems.iloc[:,1]]
print(poem_tokens[0])

['we', "'d", 'like', 'to', 'talk', 'with', 'you', 'about', 'fear', 'they', 'said', 'so', 'many', 'people', 'live', 'in', 'fear', 'these', 'days', 'they', 'drove', 'up', 'all', 'four', 'of', 'them', 'in', 'a', 'small', 'car', 'nice', 'boy', 'they', 'said', 'beautiful', 'dogs', 'they', 'said', 'so', 'friendly', 'the', 'man', 'ahead', 'of', 'the', 'woman', 'the', 'other', 'two', 'waiting', 'in', 'the', 'drive', 'i', 'was', 'outside', 'digging', 'up', 'the', 'garden', 'no', 'one', 'home', 'i', 'said', 'what', 'are', 'you', 'selling', 'anyway', 'i', "'m", 'not', 'interested', 'i', 'said', 'well', 'you', 'have', 'a', 'nice', 'day', 'they', 'said', 'here', "'s", 'our', 'card', 'there', "'s", 'a', 'phone', 'number', 'you', 'can', 'call', 'anytime', 'any', 'other', 'houses', 'down', 'this', 'road', 'anyone', 'else', 'live', 'here', 'we', "'d", 'like', 'to', 'talk', 'to', 'them', 'about', 'living', 'in', 'fear']


In [7]:
vocab = set()
for tokens in poem_tokens:
    for token in tokens:
        vocab.add(token)
print(len(vocab))

54838


In [8]:
vocab_count = {}
for tokens in poem_tokens:
    for token in tokens:
        if token not in vocab_count:
            vocab_count[token] = 1
        else:
            vocab_count[token] += 1
print(len(vocab_count))

54838


In [9]:
valid_vocab = set()
for k, v in vocab_count.items():
    if v > 10:
        valid_vocab.add(k)
print(len(valid_vocab))

8080


In [11]:
def valid_line(tokens, min_len, max_len, vocab):
    if len(tokens) < 2:
        return False
    for token in tokens:
        if token.isalnum() and token not in vocab:
            return False
        for c in token:
            if ord(c) >= 128:
                return False
    return True

poem_lines = []
for poem in poems.iloc[:,1]:
    temp_lines = poem.split('\n')
    for line in temp_lines:
        tokens = word_tokenize(line)
        if valid_line(tokens, 5, 20, valid_vocab): # valid_vocab vs words
            poem_lines.append(tokens)

In [12]:
print(len(poem_lines))

132788


In [13]:
def shorten(lines, minlen, maxlen, vocab):
    newlines = []
    for line in lines: 
        if len(line) < minlen or len(line) > maxlen:
            continue
        for w in line:
            if w not in vocab:
                continue
        newlines.append(line)
    return newlines

In [20]:
print(len(poem_lines))
cut = 120000
train = poem_lines[:cut]
valid = poem_lines[cut:]
print(len(valid))

132788
12788


In [24]:
def list_to_line(l):
    output = ""
    for w in l:
        output += w + " "
    return output

print(list_to_line(train[15]))

for here the calendar breeds nights 


In [25]:
with open("data/ptb.train.txt", "w") as file:
    for s in train:
        file.write(list_to_line(s))
        file.write('\n')
with open("data/ptb.valid.txt", "w") as file:
    for s in valid:
        file.write(list_to_line(s))
        file.write('\n')

In [26]:
count = 0
w2i = {}
i2w = {}
previous = set()
special = ["<pad>", "<unk>", "<sos>", "<eos>"]
for token in special:
    w2i[token] = count
    i2w[count] = token
    count += 1
for tokens in poem_tokens:
    for token in tokens:
        if token not in previous:
            w2i[token] = count
            i2w[count] = token
            previous.add(token)
            count += 1
final_vocab = {'w2i':w2i, 'i2w':i2w}

In [27]:
with open('data/ptb.vocab.json', 'w') as file:
    json.dump(final_vocab, file)

In [28]:
ls data

clean_poems.csv      glove.840B.300d.txt  ptb.vocab.json
final_poems.csv      ptb.train.txt        ultimate_poems.csv
glove.6B.300d.txt    ptb.valid.txt
